<a href="https://colab.research.google.com/github/brundakushi18-pixel/Cyclone_Prediction_Project/blob/main/smeinternshala4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
!pip install transformers datasets torch scikit-learn

In [3]:
from datasets import load_dataset
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
import torch
from sklearn.metrics import accuracy_score, f1_score

In [4]:
dataset = load_dataset("imdb")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [5]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
def tokenize_data(example):
  return tokenizer(example["text"], padding="max_length", truncation=True, max_length=128)

tokenized_dataset = dataset.map(tokenize_data, batched=True)

In [7]:
tokenize_dataset = tokenized_dataset.remove_columns(["text"])
tokenize_dataset = tokenize_dataset.rename_column("label", "labels")
tokenize_dataset.set_format("torch")

In [16]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01
)

In [10]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenize_dataset["train"],
    eval_dataset=tokenize_dataset["test"]
)
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 3


wandb: You chose "Don't visualize my results"


Step,Training Loss
500,0.386800
1000,0.325800
1500,0.299500
2000,0.213400
2500,0.191800
3000,0.187800
3500,0.132000
4000,0.112100
4500,0.106500


TrainOutput(global_step=4689, training_loss=0.21263361699191155, metrics={'train_runtime': 2154.0713, 'train_samples_per_second': 34.818, 'train_steps_per_second': 2.177, 'total_flos': 4933332288000000.0, 'train_loss': 0.21263361699191155, 'epoch': 3.0})

In [12]:
predictions = trainer.predict(tokenize_dataset["test"])
preds = predictions.predictions.argmax(axis=1)
labels = predictions.label_ids
accuracy = accuracy_score(labels, preds)
f1 = f1_score(labels, preds)
print("Accuracy:", accuracy)
print("F1 Score:", f1)

Accuracy: 0.89044
F1 Score: 0.8908982274447321


In [13]:
news_dataset = load_dataset("ag_news")

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [17]:
def binary_news(example):
    example["label"] = 1 if example["label"] in [1, 2] else 0
    return example
news_dataset = news_dataset.map(binary_news)

Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [29]:
tokenized_news = news_dataset.map(tokenize_data, batched=True)
for split in tokenized_news.keys():
    tokenized_news[split] = tokenized_news[split].remove_columns(["text"])
    tokenized_news[split] = tokenized_news[split].rename_column("label", "labels")
    tokenized_news[split].set_format("torch")

In [32]:
# Reinitialize Trainer for cross-domain evaluation
news_trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=None, # Not needed for prediction only
    eval_dataset=tokenized_news["test"] # Set the evaluation dataset here
)

news_predictions = news_trainer.predict(tokenized_news["test"])
news_preds = news_predictions.predictions.argmax(axis=1)
news_labels = news_predictions.label_ids
news_accuracy = accuracy_score(news_labels, news_preds)
news_f1 = f1_score(news_labels, news_preds)
print("Cross-Domain Accuracy:", news_accuracy)
print("Cross-Domain F1 Score:", news_f1)

Cross-Domain Accuracy: 0.564078947368421
Cross-Domain F1 Score: 0.593945336438289


In [ ]:
trainer_news = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenize_dataset["train"],
    eval_dataset=tokenize_dataset["test"]
)
trainer_news.train()

Step,Training Loss
500,0.084100
1000,0.090300
1500,0.087600
2000,0.045000
